TO DO:
- set dtypes for DF columns to string or int 
- reset indexes to drop extra indexes 
- acquisition dates to year only 

In [1]:
import re
import csv
import pandas as pd
import pandas_profiling as pp

Separating MOMA artworks between old (before 1980) and new (after 1980). 

In [2]:
path = './MoMA_data/'

In [ ]:

with open(path+'Artworks.csv','r') as fin, open(path+'old_artworks.csv','w') as modern, open(path+'new_artworks.csv','w') as cont:

    m_writer = csv.writer(modern, delimiter=',' )
    c_writer = csv.writer(cont, delimiter=',' )
    m_writer.writerow(['Title','Artist','ConstituentID','ArtistBio','Nationality','BeginDate','EndDate','Gender','Date','Medium','Dimensions','CreditLine','AccessionNumber','Classification','Department','DateAcquired','Cataloged','ObjectID','URL','ThumbnailURL','Circumference (cm)','Depth (cm)','Diameter (cm)','Height (cm)','Length (cm)','Weight (kg)','Width (cm)','Seat Height (cm)','Duration (sec.)'])
    c_writer.writerow(['Title','Artist','ConstituentID','ArtistBio','Nationality','BeginDate','EndDate','Gender','Date','Medium','Dimensions','CreditLine','AccessionNumber','Classification','Department','DateAcquired','Cataloged','ObjectID','URL','ThumbnailURL','Circumference (cm)','Depth (cm)','Diameter (cm)','Height (cm)','Length (cm)','Weight (kg)','Width (cm)','Seat Height (cm)','Duration (sec.)'])


    for row in csv.reader(fin, delimiter=','):

        begD = row[5]
        endD = row[6]
        if  re.match('.*\d\d\d\d.*', row[8]):
            year = re.findall(r'\d\d\d\d', row[8])
            row[8] = str(year[0])
            if int(year[0]) < 1980:
        
                m_writer.writerow(row) 
            else:
                c_writer.writerow(row)
        
        elif "century" in str(row[8]):
            m_writer.writerow(row) 

In [ ]:
#remove columns we don't need and turn csv to df 
modern = pd.read_csv(path+'old_artworks.csv')
modern = modern[['Title','Artist','ConstituentID','Date','Medium','Classification','Department','DateAcquired','URL','ThumbnailURL']]
modern = modern.fillna('missing', inplace=True)
modern.to_csv(path+'old_artworks.csv')

contemporary = pd.read_csv(path+'cont_artworks.csv')
contemporary = contemporary[['Title','Artist','ConstituentID','Date','Medium','Classification','Department','DateAcquired','URL','ThumbnailURL']]
contemporary.fillna('missing', inplace=True)
contemporary.to_csv(path+'cont_artworks.csv')
contemporary.head()

In [ ]:
#create pickles for each dempartment in modern csv

architecture_design = modern[modern['Department'] == "Architecture & Design"]
architecture_design.to_pickle(f'./MoMA_data/pickle/architecture_design_mod.pkl')

architecture_design_img  = modern[modern['Department'] == "Architecture & Design - Image Archive"]
architecture_design_img.to_pickle(f'./MoMA_data/pickle/architecture_design_img_mod.pkl')

draws_prints= modern[modern['Department'] == "Drawings & Prints"]
draws_prints.to_pickle(f'./MoMA_data/pickle/draws_prints_mod.pkl')

films= modern[modern['Department'] == "Film"]
films.to_pickle(f'./MoMA_data/pickle/films_mod.pkl')

fluxus= modern[modern['Department'] == "Fluxus Collection"]
fluxus.to_pickle(f'./MoMA_data/pickle/fluxus_mod.pkl')

media_perf= modern[modern['Department'] == "Media and Performance"]
media_perf.to_pickle(f'./MoMA_data/pickle/media_perf_mod.pkl')

painting_sculp= modern[modern['Department'] == "Painting & Sculpture"]
painting_sculp.to_pickle(f'./MoMA_data/pickle/paint_sculp_mod.pkl')

photo= modern[modern['Department'] == "Photography"]
photo.to_pickle(f'./MoMA_data/pickle/photo_mod.pkl')

In [ ]:
architecture_design = contemporary[contemporary['Department'] == "Architecture & Design"]
architecture_design.to_pickle(f'./MoMA_data/pickle/architecture_design_cont.pkl')

architecture_design_img  = contemporary[contemporary['Department'] == "Architecture & Design - Image Archive"]
architecture_design_img.to_pickle(f'./MoMA_data/pickle/architecture_design_img_cont.pkl')

draws_prints= contemporary[contemporary['Department'] == "Drawings & Prints"]
draws_prints.to_pickle(f'./MoMA_data/pickle/draws_prints_cont.pkl')

films= contemporary[contemporary['Department'] == "Film"]
films.to_pickle(f'./MoMA_data/pickle/films_cont.pkl')

fluxus= contemporary[contemporary['Department'] == "Fluxus Collection"]
fluxus.to_pickle(f'./MoMA_data/pickle/fluxus_cont.pkl')

media_perf= contemporary[contemporary['Department'] == "Media and Performance"]
media_perf.to_pickle(f'./MoMA_data/pickle/media_perf_cont.pkl')

painting_sculp= contemporary[contemporary['Department'] == "Painting & Sculpture"]
painting_sculp.to_pickle(f'./MoMA_data/pickle/paint_sculp_cont.pkl')

photo= contemporary[contemporary['Department'] == "Photography"]
photo.to_pickle(f'./MoMA_data/pickle/photo_cont.pkl')

In [ ]:
#clean gender description format to M, F, or NB

def clean_gender(file, new_file):
    for row in csv.reader(file, delimiter=','):
            a_writer = csv.writer(new_file, delimiter=',' )
            if re.match(r'[M|m]ale', row[4]):
                row[4]= 'M'
                a_writer.writerow(row)
            elif re.match(r'[F|f]emale', row[4]):
                row[4]= 'F'
                a_writer.writerow(row)
            elif re.match(r'Non-[B|b]inary', row[4]):
                row[4]= 'NB'
                a_writer.writerow(row)

In [ ]:
#run cleaning on MOMA artist db then pickle it 
with open(path+'Artists.csv','r') as artists,  open(path+'cleaned_artists.csv','w') as new_artists:
    a_writer = csv.writer(new_artists, delimiter=',' )
    a_writer.writerow(['ConstituentID','DisplayName','ArtistBio','Nationality','Gender','BeginDate','EndDate','WikiQID','ULAN'])
    new_artist= clean_gender(artists, new_artists)

new_artists= pd.read_csv(path+'cleaned_artists.csv')
new_artists.fillna('missing', inplace=True)
new_artists.to_pickle(f'./MoMA_data/pickle/artists.pkl')


Report

In [ ]:
#source file 
original_doc = pd.read_csv(path+'Artworks.csv')
pp.ProfileReport(original_doc)

In [ ]:
#modern/contemporary subsets  
pp.ProfileReport(modern)
pp.ProfileReport(contemporary)

In [ ]:
#cleaned artists 
new_artists= pd.read_csv(path+'cleaned_artists.csv')
pp.ProfileReport(new_artists)